In [ ]:
!pip install sagemaker --upgrade --user

### Training and tuning a Keras CNN on Fashion-MNIST

Fashion-MNIST is a dataset of Zalando's article images—consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes. We intend Fashion-MNIST to serve as a direct drop-in replacement for the original MNIST dataset for benchmarking machine learning algorithms. It shares the same image size and structure of training and testing splits.

https://github.com/zalandoresearch/fashion-mnist/

In this notebook, we'll first build a **custom container** training a simple CNN built with Keras.

Then, we'll apply Automatic Model Tuning to find the best parameters for this CNN.

In [ ]:
from IPython.display import Image
Image("fashion-mnist-sprite.png")

In [ ]:
import sagemaker

sess    = sagemaker.Session()
role    = sagemaker.get_execution_role()
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region  = sess.boto_session.region_name

## Dockerfile for the custom container

In [ ]:
!cat Dockerfile.gpu

In [ ]:
# Copy Dockerfiles
!cp Dockerfile.* build/

In [ ]:
# Copy training script
!cp mnist_cnn.py build/

## Create and login to a repository in ECR

### GPU settings

In [ ]:
repo_name = 'keras-tf-gpu' # ECR repository
image_tag = 'keras-tf-gpu-py3' # ECR image tag
base_job_name = 'keras-tf-mnist-cnn' # SageMaker training prefix

%env dockerfile Dockerfile.gpu

train_instance_type = 'ml.p3.2xlarge'
gpu_count           = 1
batch_size          = 128*gpu_count

### Common settings

In [ ]:
%env account {account}
%env region {region}
%env repo_name {repo_name}
%env image_tag {image_tag}

### Create repository and login

In [ ]:
%%sh

aws ecr describe-repositories --repository-names $repo_name > /dev/null 2>&1
if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name $repo_name > /dev/null
fi

$(aws ecr get-login --region $region --no-include-email)

## Build and tag Docker image

In [ ]:
%cd build
!docker build -t $image_tag -f $dockerfile .
%cd ..    

In [ ]:
!docker tag $image_tag $account.dkr.ecr.$region.amazonaws.com/$repo_name:latest

In [ ]:
!docker images

In [ ]:
# It's probably a good idea to inspect your container before pushing it :)
# !docker -it /bin/bash $CONTAINER

## Push Docker image to ECR

In [ ]:
!docker push $account.dkr.ecr.$region.amazonaws.com/$repo_name:latest

## Upload Fashion-MNIST data to S3

In [ ]:
local_directory = 'data'
prefix          = repo_name+'/input'

train_input_path      = sess.upload_data(local_directory+'/train/',      key_prefix=prefix+'/train')
validation_input_path = sess.upload_data(local_directory+'/validation/', key_prefix=prefix+'/validation')

## Train with the custom container

In [ ]:
metric_definitions = [
    {'Name': 'val_acc', 'Regex': 'val_acc: ([0-9\\.]+)'},
    {'Name': 'val_loss', 'Regex': 'val_loss: ([0-9\\.]+)'}
]

In [ ]:
output_path = 's3://{}/{}/output'.format(sess.default_bucket(), repo_name)
image_name  = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account, region, repo_name)

print(output_path)
print(image_name)

estimator = sagemaker.estimator.Estimator(
                       image_name=image_name,
                       metric_definitions=metric_definitions,
                       base_job_name=base_job_name,
                       role=role, 
                       train_instance_count=1, 
                       train_instance_type=train_instance_type,
                       output_path=output_path,
                       sagemaker_session=sess)

In [ ]:
estimator.set_hyperparameters(epochs=100, gpus=gpu_count, batch_size=batch_size)

estimator.fit({'training': train_input_path, 'validation': validation_input_path})

## Define hyperparameters to optimize

In [ ]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

In [ ]:
hyperparameter_ranges = {
                         'filters1'   : IntegerParameter(32, 128),
                         'filters2'   : IntegerParameter(64, 256),
                         'dropout1'   : ContinuousParameter(0.01, 0.8),
                         'dropout2'   : ContinuousParameter(0.01, 0.8),
                         'fc1'         : IntegerParameter(128, 1024),
                         'fc2'         : IntegerParameter(64, 512),
                         'dropout_fc1' : ContinuousParameter(0.2, 0.8),
                         'dropout_fc2' : ContinuousParameter(0.2, 0.8)
                        }

## Set static hyperparameters

In [ ]:
estimator.set_hyperparameters(epochs=100, gpus=gpu_count, batch_size=batch_size)

## Define metric to optimize for

In [ ]:
objective_metric_name = 'val_acc'
objective_type = 'Maximize'
metric_definitions = [{'Name': 'val_acc',
                       'Regex': 'Best val_acc: ([0-9\\.]+)'}]

## Configure and launch tuning job

In [ ]:
tuner = HyperparameterTuner(estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=30,
                            max_parallel_jobs=3,
                            objective_type=objective_type)

In [ ]:
tuner.fit({'training': train_input_path, 'validation': validation_input_path})

## Monitor the tuning job

In [ ]:
import boto3, pprint
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

sm = boto3.client('sagemaker')
pp = pprint.PrettyPrinter(indent=4)

In [ ]:
job_name = tuner._current_job_name
resp = sm.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=job_name)

pp.pprint(resp['HyperParameterTuningJobStatus'])
pp.pprint(resp['ObjectiveStatusCounters'])

In [ ]:
import pandas as pd

# Show completed jobs sorted by descending accuracy
stats = tuner.analytics().dataframe()
stats = stats[stats['TrainingJobStatus'] != 'InProgress']
stats = stats.sort_values(by=['FinalObjectiveValue'], ascending=False)
stats.drop(columns=['TrainingElapsedTimeSeconds', 'TrainingStartTime', 
                    'TrainingEndTime', 'TrainingJobStatus']).head(30)

In [ ]:
stats = stats[stats['FinalObjectiveValue'] > 0.943]

## Plotting accuracy vs. time

In [ ]:
from sklearn.linear_model import LinearRegression

plt.title('Accuracy vs. time')
plot_x = stats['TrainingEndTime'].tolist()
# convert datetimes to timestamps and reshape list to a column (for the LR model)
plot_x = np.array([x.timestamp() for x in plot_x]).reshape(-1, 1) 
plot_y = stats['FinalObjectiveValue'].tolist()
plt.scatter(plot_x, plot_y, color="blue")

# Fit a linear regression model, predict values and plot line
lr = LinearRegression()
lr.fit(plot_x, plot_y)
plot_y_new = lr.predict(plot_x)
plt.plot(plot_x, plot_y_new, color='red')
# Hopefully the line is going up!
print(lr.coef_)

## Plotting accuracy vs. dropout

In [ ]:
plt.xticks(np.arange(0.2, 0.9, 0.1))
plt.title('Accuracy vs. CNN layer 1 dropout')
plt.scatter(stats['dropout1'].tolist(), stats['FinalObjectiveValue'].tolist(), color='red')
plt.show()
plt.title('Accuracy vs. CNN layer 2 dropout')
plt.scatter(stats['dropout2'].tolist(), stats['FinalObjectiveValue'].tolist(), color='green')
plt.show()
plt.title('Accuracy vs. FC layer 1 dropout')
plt.scatter(stats['dropout_fc1'].tolist(), stats['FinalObjectiveValue'].tolist(), color='blue')
plt.show()
plt.title('Accuracy vs. FC layer 2 dropout')
plt.scatter(stats['dropout_fc2'].tolist(), stats['FinalObjectiveValue'].tolist(), color='orange')
plt.show()

## Plotting accuracy vs. CNN filters

In [ ]:
plt.title('Accuracy vs. CNN layer 1 filters')
plt.scatter(stats['filters1'].tolist(), stats['FinalObjectiveValue'].tolist(), color='red')
plt.show()
plt.title('Accuracy vs. CNN layer 2 filters')
plt.scatter(stats['filters2'].tolist(), stats['FinalObjectiveValue'].tolist(), color='green')
plt.show()

## Plotting accuracy vs. fully-connected layer

In [ ]:
plt.title('Accuracy vs. FC layer 1 width')
plt.scatter(stats['fc1'].tolist(), stats['FinalObjectiveValue'].tolist(), color='red')
plt.show()
plt.title('Accuracy vs. FC layer 2 width')
plt.scatter(stats['fc2'].tolist(), stats['FinalObjectiveValue'].tolist(), color='green')
plt.show()

## Print hyperparameters for the best training job

In [ ]:
resp = sm.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=job_name)

best = resp['BestTrainingJob']
pp.pprint(best['TrainingJobName'])
print("%.2f%%" % (100.0*best['FinalHyperParameterTuningJobObjectiveMetric']['Value']))
pp.pprint(best['TunedHyperParameters'])

## Deploy best model

In [ ]:
endpoint_name = 'best-mnist-cnn'

predictor = tuner.deploy(initial_instance_count=1, 
             instance_type='ml.m4.xlarge', 
             endpoint_name=endpoint_name)

In [ ]:
sagemaker.delete_endpoint(endpoint_name=endpoint_name)